In [1]:
import os
import openai
import sys

from dotenv import load_dotenv, find_dotenv

from glob import glob
from os.path import join

from tqdm import tqdm

from langchain.document_loaders import PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

from langchain.llms import OpenAI

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, ConversationalRetrievalChain

from langchain.memory import ConversationBufferMemory



In [2]:
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

# Creating ChromaDB

In [3]:
files_dir = 'data'

# Use a função glob para listar todos os arquivos PDF no diretório
pdf_files = glob(f'{files_dir}/*.pdf')

docs=[]

for arquivo in tqdm(pdf_files):
    print(arquivo)

    loader = PyPDFLoader(arquivo)
    docs.extend(loader.load())

  0%|          | 0/34 [00:00<?, ?it/s]

data\35-GHz Barium Hexaferrite or PDMS Composite-Based Millimeter-Wave Circulators for 5G Applications.pdf


  3%|▎         | 1/34 [00:01<00:33,  1.03s/it]

data\A comprehensive survey on machine learning for networking - Evolution, applications and research opportunities.pdf


  6%|▌         | 2/34 [00:06<01:56,  3.63s/it]

...

 94%|█████████▍| 32/34 [02:13<00:04,  2.30s/it]

data\Towards Supporting Intelligence in 5G-6G Core.pdf


 97%|█████████▋| 33/34 [02:14<00:01,  1.76s/it]

data\Understanding Mobile Traffic Patterns of Large Scale Cellular Towers in Urban Environment.pdf


100%|██████████| 34/34 [02:18<00:00,  4.09s/it]


In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(docs)

In [5]:
#!rm -Rf /Users/patrick/Documents/Pessoal/paper-rag/chroma/

embedding = OpenAIEmbeddings()
persist_directory = 'chroma_full'

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

c:\Users\PatrickLuizdeAraujo\anaconda3\envs\watsonxai\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


# Model

In [6]:
llm = OpenAI(temperature=0)  # Model used

c:\Users\PatrickLuizdeAraujo\anaconda3\envs\watsonxai\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


# Template

In [7]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you can't make a answer with context, just say that you don't know, don't try to make up an answer. Give the document name from where the information is extracted
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Compression retrieval method configuration

In [8]:
# Wrap our vectorstore
compressor = LLMChainExtractor.from_llm(llm)  # uses an LLMChain to extract from each document only the statements that are relevant to the query.

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")  # Retriever method using MMR retrieval to ensure variability in the answer
)

# Chain assembling

In [9]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [10]:
question = "What is Nnwdaf_AnalyticsSubscriptionservice?"
result = qa_chain({"query": question})

print(result["result"])
print('\n')

for source in result['source_documents']:
    print ('Source: ', source.metadata['source'].split('/')[-1])
    print('Page: ', source.metadata['page'])
    print('\n')

c:\Users\PatrickLuizdeAraujo\anaconda3\envs\watsonxai\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Users\PatrickLuizdeAraujo\anaconda3\envs\watsonxai\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\PatrickLuizdeAraujo\anaconda3\envs\watsonxai\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\PatrickLuizdeAraujo\anaconda3\envs\watsonxai\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\PatrickLuizdeAraujo\anaconda3\envs\watsonx

 The Nnwdaf_AnalyticsSubscriptionservice is a service provided by the NWDAF (Network Data Analytics Function) that notifies the NF (Network Function) consumer instance of all analytics subscribed to the specific NWDAF service. This information is extracted from the document "NWDAF Services and Interfaces" which describes the different services provided by the NWDAF.


Source:  data\Towards Supporting Intelligence in 5G-6G Core.pdf
Page:  1


Source:  data\Resource Sharing Efficiency in Network_Slicing.pdf
Page:  5


Source:  data\Improving traffic forecasting for 5G core network scalability - A machine learning approach.pdf
Page:  5


